## Welcome to our EECS 442 project: Colorization of Iconic Sports Photos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a EECS442 folder and put all the files under HW5 folder, then 'EECS442/HW5'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS442_Project'

In [ ]:
import os
import sys

GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)


In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/MyDrive/EECS442_Project


In [ ]:
%cd drive/MyDrive/EECS442_Project

/content/drive/.shortcut-targets-by-id/1ckixt1SXyEJ89NxG9GoD7MfjvT8tKVu_/EECS442_Project


Import all the packages needed

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools
from matplotlib import image
import glob as glob
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from skimage import color
from skimage.color import rgb2lab, lab2rgb

### DataLoader:

In [ ]:
class RecolorData(Dataset):
  def __init__(self, root_dir, size=128, split='train', transform=None):
    """
    Args:
        root_dir: the directory of the dataset
        split: "train", "valid", or "test"
        transform: pytorch transformations.
    """

    self.transform = transform

    self.files = glob.glob(os.path.join(root_dir, split + str(size), '*.jpg'))

  def __len__(self):
    return len(self.files)

  def __getitem__(self, idx):
    img = Image.open(self.files[idx])
    img = color.rgb2lab(img).astype("float32")
    img = np.asarray(img)
    if self.transform:
        img = self.transform(img)
    return {'L': img[0:1, :, :], 'ab': img[1:3, :, :]}

In [ ]:
%ls
%cd Downloads/

code_1.87.2-1709912201_amd64.deb
dataset128/
dataset128.zip
Spinnaker/
spinnaker-4.0.0.116-amd64-pkg-22.04.tar.gz
spinnaker_python-4.0.0.116-cp310-cp310-linux_x86_64.tar.gz
[Errno 2] No such file or directory: 'Downloads/'
/home/mcity/Downloads


/home/mcity/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
# Define transformation to apply to the images
transform = transforms.Compose([
    #transforms.Resize((256, 256)),  # Resize images to a fixed size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

# Define the directory containing your image folders
data_dir = 'dataset128'

# Load the dataset using ImageFolder
train_dataset = RecolorData(data_dir, split="train", transform=transform)
valid_dataset = RecolorData(data_dir, split="valid", transform=transform)
test_dataset = RecolorData(data_dir, split="test", transform=transform)

print(len(train_dataset))

batch_size = 16

# Create a DataLoader to iterate over the dataset
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print('Number of training images {}, number of validation images {}, number of testing images {}'.format(len(train_dataset), len(valid_dataset), len(test_dataset)))

data = next(iter(train_dl))
Ls, abs_ = data['L'], data['ab']
print(Ls.shape, abs_.shape)
print(len(train_dl), len(val_dl))

13492
Number of training images 13492, number of validation images 500, number of testing images 500
torch.Size([16, 1, 128, 128]) torch.Size([16, 2, 128, 128])
844 32


### U-Net

In [ ]:
from math import e
class Unet(nn.Module):
    def __init__(self, input_c=1, output_c=2, n_down=8, num_filters=64):
        super().__init__()
        self.layer1_d = nn.Sequential(
            nn.Conv2d(1, 32, 4, stride=2, padding=1),
        )
        self.layer2_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64)
        )
        self.layer3_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128)
        )
        self.layer4_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256)
        )
        self.layer5_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),
            nn.BatchNorm2d(512)
        )
        self.layer6_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(512, 1024, 4, stride=2, padding=1),
            nn.BatchNorm2d(1024)
        )
        self.layer7_d = nn.Sequential(
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(1024, 1024, 4, stride=2, padding=1),
        )

        self.layer7_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024)
        )
        self.layer6_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(2048, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512)
        )
        self.layer5_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256)
        )
        self.layer4_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(512, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128)
        )
        self.layer3_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64)
        )
        self.layer2_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(128, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32)
        )
        self.layer1_u = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
        self.up_layers = [self.layer1_u, self.layer2_u, self.layer3_u, self.layer4_u, self.layer5_u, self.layer6_u, self.layer7_u]
        self.down_layers = [self.layer1_d, self.layer2_d, self.layer3_d, self.layer4_d, self.layer5_d, self.layer6_d, self.layer7_d]

        # self.init_weights()

    def init_weights(self, mean, std):
      for m in self._modules:
        if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
          m.weight.data.normal_(mean, std)
          m.bias.data.zero_()


    def forward(self, input):
        #down (encode)
        e1 = self.layer1_d(input)
        e2 = self.layer2_d(e1)
        e3 = self.layer3_d(e2)
        e4 = self.layer4_d(e3)
        e5 = self.layer5_d(e4)
        e6 = self.layer6_d(e5)
        e7 = self.layer7_d(e6)

        #up (decode)
        d7 = self.layer7_u(e7)
        d7 = torch.cat([d7, e6], 1)
        d6 = self.layer6_u(d7)
        d6 = torch.cat([d6, e5], 1)
        d5 = self.layer5_u(d6)
        d5 = torch.cat([d5, e4], 1)
        d4 = self.layer4_u(d5)
        d4 = torch.cat([d4, e3], 1)
        d3 = self.layer3_u(d4)
        d3 = torch.cat([d3, e2], 1)
        d2 = self.layer2_u(d3)
        d2 = torch.cat([d2, e1], 1)
        d1= self.layer1_u(d2)

        return d1

### Discriminator

Patch Discriminator Bx3x128x128 to Bx1x15x15

In [ ]:
class PatchDiscriminator(nn.Module):
  # initializers
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
    self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
    self.bn2 = nn.BatchNorm2d(128)
    self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
    self.bn3 = nn.BatchNorm2d(256)
    self.conv4 = nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=1)
    #self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1)
    #self.bn4 = nn.BatchNorm2d(512)
    #self.conv5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)

  # weight_init - Doesn't work as normal_init is not defined
  def weight_init(self, mean, std):
    for m in self._modules:
      normal_init(self._modules[m], mean, std)


  def init_weights(self, mean, std):
      for m in self._modules:
        if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
          m.weight.data.normal_(mean, std)
          m.bias.data.zero_()


  # forward method
  def forward(self, input):

    x1 = F.leaky_relu(self.conv1(input))
    x2 = F.leaky_relu(self.bn2(self.conv2(x1)), negative_slope=0.1)
    x3 = F.leaky_relu(self.bn3(self.conv3(x2)), negative_slope=0.1)
    x4 = self.conv4(x3)
    #x4 = F.leaky_relu(self.bn4(self.conv4(x3)), negative_slope=0.2)
    #x4 = F.sigmoid(self.conv4(x3))

    return x4

### GAN Loss

Not currently used

In [ ]:
class GANLoss(nn.Module):
    def __init__(self, gan_mode='vanilla', real_label=1.0, fake_label=0.0):
        super().__init__()
        self.register_buffer('real_label', torch.tensor(real_label))
        self.register_buffer('fake_label', torch.tensor(fake_label))
        if gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode == 'lsgan':
            self.loss = nn.MSELoss()

    def get_labels(self, preds, target_is_real):
        if target_is_real:
            labels = self.real_label
        else:
            labels = self.fake_label
        return labels.expand_as(preds)

    def __call__(self, preds, target_is_real):
        labels = self.get_labels(preds, target_is_real)
        loss = self.loss(preds, labels)
        return loss

### Summary

In [ ]:
G = Unet().cuda()
D = PatchDiscriminator().cuda()
summary(G, (1, 128, 128))
summary(D, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             544
         LeakyReLU-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 64, 32, 32]          32,832
       BatchNorm2d-4           [-1, 64, 32, 32]             128
         LeakyReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6          [-1, 128, 16, 16]         131,200
       BatchNorm2d-7          [-1, 128, 16, 16]             256
         LeakyReLU-8          [-1, 128, 16, 16]               0
            Conv2d-9            [-1, 256, 8, 8]         524,544
      BatchNorm2d-10            [-1, 256, 8, 8]             512
        LeakyReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 512, 4, 4]       2,097,664
      BatchNorm2d-13            [-1, 512, 4, 4]           1,024
        LeakyReLU-14            [-1, 51

### Helpers

In [ ]:
def lab_to_rgb(L, ab):
    """
    Takes a batch of images
    """

    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)

def visualize(G, data, save=True):
    L = data['L']
    real_color = data['ab']
    fake_color = G(L)
    fake_imgs = lab_to_rgb(L, fake_color)
    real_imgs = lab_to_rgb(L, real_color)
    fig = plt.figure(figsize=(15, 8))
    for i in range(5):
        ax = plt.subplot(3, 5, i + 1)
        ax.imshow(L[i][0].cpu(), cmap='gray')
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 5)
        ax.imshow(fake_imgs[i])
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 10)
        ax.imshow(real_imgs[i])
        ax.axis("off")
    plt.show()
    if save:
        fig.savefig(f"colorization_{time.time()}.png")

def log_results(loss_meter_dict):
    for loss_name, loss_meter in loss_meter_dict.items():
        print(f"{loss_name}: {loss_meter.avg:.5f}")

In [ ]:
'''
#Sample Output used for visualization
test = val_dl.__iter__().__next__()
img_size = 256
fixed_y_ = test['ab'].cuda()
fixed_x_ = test['L'].cuda()
print(len(train_dl))
print(len(val_dl))
print(fixed_y_.shape)

# plot sample image
fig, axes = plt.subplots(2, 2)
axes = np.reshape(axes, (4, ))
for i in range(4):
  example = train_dl.__iter__().__next__()[i].numpy().transpose((1, 2, 0))
  mean = np.array([0.5, 0.5, 0.5])
  std = np.array([0.5, 0.5, 0.5])
  example = std * example + mean
  axes[i].imshow(example)
  axes[i].axis('off')
plt.show()
'''

"\n#Sample Output used for visualization\ntest = val_dl.__iter__().__next__()\nimg_size = 256\nfixed_y_ = test['ab'].cuda()\nfixed_x_ = test['L'].cuda()\nprint(len(train_dl))\nprint(len(val_dl))\nprint(fixed_y_.shape)\n\n# plot sample image\nfig, axes = plt.subplots(2, 2)\naxes = np.reshape(axes, (4, ))\nfor i in range(4):\n  example = train_dl.__iter__().__next__()[i].numpy().transpose((1, 2, 0))\n  mean = np.array([0.5, 0.5, 0.5])\n  std = np.array([0.5, 0.5, 0.5])\n  example = std * example + mean\n  axes[i].imshow(example)\n  axes[i].axis('off')\nplt.show()\n"

In [ ]:
def process_image(img):
  return (img.cpu().data.numpy().transpose(1, 2, 0) + 1) / 2

def show_result(G, x_, y_, num_epoch):
  predict_images = G(x_)

  fig, ax = plt.subplots(x_.size()[0], 3, figsize=(6,10))

  for i in range(x_.size()[0]):
    ax[i, 0].get_xaxis().set_visible(False)
    ax[i, 0].get_yaxis().set_visible(False)
    ax[i, 1].get_xaxis().set_visible(False)
    ax[i, 1].get_yaxis().set_visible(False)
    ax[i, 2].get_xaxis().set_visible(False)
    ax[i, 2].get_yaxis().set_visible(False)
    ax[i, 0].cla()
    ax[i, 0].imshow(process_image(x_[i]))
    ax[i, 1].cla()
    ax[i, 1].imshow(process_image(predict_images[i]))
    ax[i, 2].cla()
    ax[i, 2].imshow(process_image(y_[i]))

  plt.tight_layout()
  label_epoch = 'Epoch {0}'.format(num_epoch)
  fig.text(0.5, 0, label_epoch, ha='center')
  label_input = 'Input'
  fig.text(0.18, 1, label_input, ha='center')
  label_output = 'Output'
  fig.text(0.5, 1, label_output, ha='center')
  label_truth = 'Ground truth'
  fig.text(0.81, 1, label_truth, ha='center')

  plt.show()

# Helper function for counting number of trainable parameters.
def count_params(model):
  '''
  Counts the number of trainable parameters in PyTorch.
  Args:
      model: PyTorch model.
  Returns:
      num_params: int, number of trainable parameters.
  '''
  num_params = sum([item.numel() for item in model.parameters() if item.requires_grad])
  return num_paramsG_100

In [ ]:
# Hint: you could use following loss to complete following function
BCE_loss = nn.BCEWithLogitsLoss().cuda()
L1_loss = nn.L1Loss().cuda()

def train(G, D, num_epochs = 20):
  hist_D_losses = []
  hist_G_losses = []
  hist_G_L1_losses = []

  lr=0.0002
  beta=0.5
  beta2=0.999
  G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(beta, beta2))
  D_optimizer = optim.Adam(D.parameters(), lr=0.5*lr, betas=(beta, beta2))


  print('training start!')
  start_time = time.time()
  for epoch in range(num_epochs):
    print('Start training epoch %d' % (epoch + 1))
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    num_iter = 0
    for batch in train_dl:
      x_ = batch['L'].cuda()
      y_ = batch['ab'].cuda()

      #optimize D
      D_optimizer.zero_grad()

      #real data
      D_real_preds = D(torch.cat([x_, y_], dim=1))
      D_real_loss = BCE_loss(D_real_preds, torch.ones_like(D_real_preds).cuda())

      #fake data
      fake_data = G(x_)
      D_fake_preds = D(torch.cat([x_,fake_data.detach()],dim=1))
      D_fake_loss = BCE_loss(D_fake_preds, torch.zeros_like(D_fake_preds).cuda())

      #update D
      loss_D = (D_real_loss + D_fake_loss) / 2
      loss_D.backward()
      D_optimizer.step()


      #optimize G
      G_optimizer.zero_grad()

      #Loss
      D_fake_preds = D(torch.cat([x_,fake_data.detach()],dim=1))
      G_BCE_loss = BCE_loss(D_fake_preds, torch.ones_like(D_fake_preds).cuda())
      G_L1_loss = L1_loss(fake_data,y_)
      L1_lambda = 100
      loss_G = G_BCE_loss + L1_lambda*G_L1_loss

      loss_G.backward()
      G_optimizer.step()

      hist_D_losses.append(loss_D.item())
      hist_G_losses.append(G_BCE_loss.item())
      hist_G_L1_losses.append(G_L1_loss.item())
      D_losses.append(loss_D.item())
      G_losses.append(loss_G.item())
      num_iter += 1



    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time

    print('[%d/%d] - using time: %.2f seconds' % ((epoch + 1), num_epochs, per_epoch_ptime))
    print('loss of discriminator D: %.3f' % (torch.mean(torch.FloatTensor(D_losses))))
    print('loss of generator G: %.3f' % (torch.mean(torch.FloatTensor(G_losses))))
    if epoch == 0 or (epoch + 1) % 5 == 0:
      with torch.no_grad():
        visualize(G, batch, save=False)

  end_time = time.time()
  total_ptime = end_time - start_time

  return hist_D_losses, hist_G_losses, hist_G_L1_losses

In [ ]:
G_100 = Unet()
D_100 = PatchDiscriminator()
G_100.init_weights(mean=0.0, std=0.02)
D_100.init_weights(mean=0.0, std=0.02)
G_100.cuda()
D_100.cuda()
G_100.train()
D_100.train()

PatchDiscriminator(
  (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
)

### Begin Training

In [ ]:
hist_D_100_losses, hist_G_100_BCE_losses, hist_G_100_L1_losses = train(G_100,D_100,num_epochs=20)

training start!
Start training epoch 1
